## DEPENDANT ON SENTIMENTS

In [1]:
import datetime
import pandas as pd
import numpy as np

In [2]:
def getTargets():
    targets = pd.read_csv('targetSp.csv')
    return targets.Symbol.to_numpy()

In [3]:
def getSentiments():
    return pd.read_csv('data/' +(datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d') + '/sentiments.csv')

In [4]:
def getScore(target, sentiments):
    tweets = sentiments[sentiments['type'] == 'tweet']
    tweets = tweets[tweets['target'] == target]
    headlines = sentiments[sentiments['type'] == 'headline']
    headlines = headlines[headlines['target'] == target]
    tweets['score'] = tweets.apply(lambda x: posNegNeut(x) , axis=1)
    headlines['score'] = headlines.apply(lambda x: posNegNeut(x) , axis=1)
    return{'target': target, 'tweets': tweets['score'].mean(), 'headlines':headlines['score'].mean()}
        
def posNegNeut(x):
    if x['label'] == 'neutral':
        return 0 
    elif x['label'] == 'negative':
        return x['score']*-1
    else:
        return x['score']

In [5]:
def buyShares(scores):
    scores['combinedBuy'] = scores.apply(lambda x: shareConf(x, 'combined'), axis=1)
    scores['tweetBuy'] = scores.apply(lambda x: shareConf(x, 'tweets'), axis=1)
    scores['headlineBuy'] = scores.apply(lambda x: shareConf(x, 'headlines'), axis=1)
    return scores

def shareConf(x, target):
    if x[target] < .1:
        return 0
    elif x[target] < .2:
        return 10
    elif x[target] < .3:
        return 20
    elif x[target] < .4:
        return 30
    elif x[target] < .5:
        return 40
    else:
        return 50

In [6]:
def runDaily(alpha):
    targets = getTargets()
    sentiments = getSentiments()
    scores = pd.DataFrame()
    for target in targets:
        scores = scores.append(getScore(target, sentiments), ignore_index=True)
    scores['combined'] = scores['tweets']*alpha + scores['headlines']*(1-alpha)
    scores['alpha'] = alpha
    scores = buyShares(scores)
    scores.to_csv('data/' +(datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d') + '/scores.csv')
    return scores

In [7]:
data = runDaily(.3)
data

,headlines,target,tweets,combined,alpha,combinedBuy,tweetBuy,headlineBuy
0,0.399639,CSCO,0.142799,0.322587,0.3,30,10,30
1,0.199933,FAST,0.025369,0.147563,0.3,10,0,10
2,0.179808,FB,0.063104,0.144797,0.3,10,0,10
3,-0.196616,KSU,0.019435,-0.131801,0.3,0,0,0
4,0.590063,ADP,0.256208,0.489907,0.3,40,20,50
5,-0.399282,GLW,0.330343,-0.180395,0.3,0,30,0
6,0.199858,EXPE,0.049752,0.154826,0.3,10,0,10
7,0.000461,MHK,0.199429,0.060151,0.3,0,10,0
8,0.204251,ALGN,0.000000,0.142976,0.3,10,0,20
9,0.199918,SEE,0.109444,0.172776,0.3,10,10,10
